# Convolution_Neural_Network

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

#Load the data
mnist = input_data.read_data_sets("MNIST/", one_hot=True)

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [2]:
def weight(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev = 0.1))

def model(w_conv1, w_conv2, w_h, w_o, keep_rate_conv, keep_rate_hid):
    conv1_o = tf.nn.relu(tf.nn.conv2d(X, w_conv1, strides = [1, 1, 1, 1], padding = 'SAME'))
    pool1_o = tf.nn.max_pool(conv1_o, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    pool1_o = tf.nn.dropout(pool1_o,keep_rate_conv)
    
    conv2_o = tf.nn.relu(tf.nn.conv2d(pool1_o, w_conv2, strides = [1, 1, 1, 1], padding = 'SAME'))
    pool2_o = tf.nn.max_pool(conv2_o, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    pool2_o = tf.nn.dropout(tf.reshape(pool2_o, [-1, w_h.get_shape().as_list()[0]]), keep_rate_conv)
    
    hidden_o = tf.nn.relu(tf.matmul(pool2_o, w_h))
    hidden_o = tf.nn.dropout(hidden_o,keep_rate_hid)
    
    return tf.matmul(hidden_o, w_o)   

In [3]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_rate_conv = tf.placeholder(tf.float32)
keep_rate_hid = tf.placeholder(tf.float32)

w_conv1 = weight([3, 3, 1, 32])
w_conv2 = weight([3, 3, 32, 64])
w_h = weight([7 * 7 * 64, 625])
w_o = weight([625, 10])

y_pred = model(w_conv1, w_conv2, w_h, w_o, keep_rate_conv, keep_rate_hid)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_pred, Y))
optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

predict_correction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(predict_correction, tf.float32))

init = tf.initialize_all_variables()

In [5]:
with tf.Session() as sess:
    sess.run(init)
    Xtr, Ytr = mnist.train.images, mnist.train.labels
    Xtr = Xtr.reshape(-1, 28, 28, 1)
    for i in range(5):
        for start, end in zip(range(0, len(Xtr), 128), range(128, len(Xtr)+1, 128)):
            optimizer.run(feed_dict = \
                          {X: Xtr[start: end], Y: Ytr[start: end], keep_rate_conv: 0.8, keep_rate_hid: 0.5})
            train_accuracy = sess.run(accuracy, feed_dict = \
                                          {X: Xtr[start: end], Y: Ytr[start: end],\
                                           keep_rate_conv: 0.8, keep_rate_hid: 0.5})
            
        print "Epoch: %d, Accuracy: %f" %(i+1, train_accuracy)
    
    print "Done"
    Xte, Yte = mnist.train.next_batch(500)
    Xte = Xte.reshape(-1, 28, 28, 1)
    print "Test Accuracy: ", sess.run(accuracy, feed_dict = {X: Xte, Y: Yte, keep_rate_conv: 1, keep_rate_hid: 1})

 Epoch: 1, Accuracy: 1.000000
Epoch: 2, Accuracy: 1.000000
Epoch: 3, Accuracy: 0.992188
Epoch: 4, Accuracy: 1.000000
Epoch: 5, Accuracy: 0.992188
Done
Test Accuracy:  0.994
